# Course: Deep Learning
# Author: Sandro Camargo <sandrocamargo@unipampa.edu.br>
# Non Linear Regression with Multi Layer Perceptron Example
# Overfitting Example
# Dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names

In [ ]:
# Import Libraries
import keras # Neural Network Library
import pandas as pd # Data Manipulation library
import numpy as np # Fast Numeric Computing library
import tensorflow as tf # Optimizers
import matplotlib.pyplot as plt # Plot library
from keras import layers # Layers to a neural network

In [ ]:
# Loading dataset
colnames=['Crim','Zn','Indus','Chas','Nox','Rm','Age','Dis','Rad','Tax','PtRatio','B','Lstat','MedV']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', sep="\s+", header=None, names=colnames)

# inspecting columns and data types from "data" dataframe
data.info()

In [ ]:
# Descriptive Statistics
data.describe()

In [ ]:
# Split inputs and outputs
output = data['MedV']
inputs = data
inputs.drop(inputs.columns[[13]], axis=1, inplace=True) # column 13 is removed, because it is the output (y)

In [ ]:
# Defining normalizing function
def normalize(dataset):
  mu = np.mean(dataset, axis = 0)
  sigma = np.std(dataset, axis = 0)
  return (dataset - mu)/sigma

In [ ]:
inputs_norm = normalize(inputs)

np.random.seed(1) # Random numbers will be ever the same
rnd = np.random.rand(len(inputs_norm)) < 0.8 # Training set will contain 80% of the data

# Creating the training dataset (80%)
train_x = inputs_norm[rnd]
train_y = output[rnd]

# Creating the validation dataset (20%)
val_x = inputs_norm[~rnd]
val_y = output[~rnd]

# Verifying dataset dimensions
print('The training dataset (inputs) dimensions are: ', train_x.shape)
print('The training dataset (outputs) dimensions are: ', train_y.shape)
print('The validation dataset (inputs) dimensions are: ', val_x.shape)
print('The validation dataset (outputs) dimensions are: ', val_y.shape)

In [ ]:
def create_and_train_model_nlayers(data_train_norm, labels_train, data_val_norm, labels_val, num_neurons, num_layers):
  # build model
  # input layer
  inputs = keras.Input(shape = data_train_norm.shape[1])
  # he initialization
  initializer = tf.keras.initializers.HeNormal()
  # first hidden layer
  dense = layers.Dense(num_neurons, activation = 'relu', kernel_initializer = initializer)(inputs)
  # customized number of layers and neurons per layer
  for i in range(num_layers - 1):
    dense = layers.Dense(num_neurons, activation = 'relu', kernel_initializer = initializer)(dense)

  # output layer
  outputs = layers.Dense(1)(dense)
  model = keras.Model(inputs = inputs, outputs = outputs, name = 'model')
  # set optimizer and loss
  opt = keras.optimizers.Adam(learning_rate = 0.001)
  model.compile(loss = 'mse', optimizer = opt, metrics = ['mse','mae'])
  # train model
  history = model.fit(
    data_train_norm, labels_train,
    epochs = 10000, verbose = 1,
    batch_size = data_train_norm.shape[0],
    validation_data = (data_val_norm, labels_val))
  # save performances
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  return hist, model

In [ ]:
hist, model = create_and_train_model_nlayers(train_x, train_y, val_x, val_y, 20, 4)

In [ ]:
model.summary()

This plot should be generated just to inspect the learning convergence.
It is expected a decreasing of the loss function value through the epochs.

In [ ]:
plt.plot(hist['mse'], label="Training MSE")
plt.plot(hist['val_mse'], label="Validation MSE")
plt.title('Training Process')
plt.ylabel('Loss Function (MSE)')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.savefig("mlp-regr-regularization.png")
plt.show()
print("Training Categorical Accuracy:", round(history.history['categorical_accuracy'][-1],4))
print("Validation Categorical Accuracy:", round(history.history['val_categorical_accuracy'][-1],4))

Similar to the previous plot, but highlighting the divergence between training and validation sets

In [ ]:
plt.plot(hist['mse'], label="Training MSE")
plt.plot(hist['val_mse'], label="Validation MSE")
plt.ylim(0,30)
plt.title('Training Process')
plt.ylabel('Loss Function (MSE)')
plt.xlabel('Epoch')
plt.legend(loc='upper center')
plt.savefig("mlp-regr-regularization-zoom.png")
plt.show()

In [ ]:
test_predictions = model.predict(val_x) # predict radon activities with the built linear regression model

plt.scatter(val_y, test_predictions, marker = 'o', c = 'blue')
plt.plot([0,55], [0,55], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Regression with MLP (Testing Set)')
plt.ylim(0, 55)
plt.xlim(0, 55)
plt.axis(True)
plt.savefig("mlp-regr-regul-testing.png")
plt.show()

print("Correlation Coefficient in testing set: %.4f" % np.corrcoef(np.transpose(test_predictions), np.transpose(val_y))[0,1])

In [ ]:
train_predictions = model.predict(train_x) # predict radom activities with the built linear regression model

plt.scatter(train_y, train_predictions, marker = 'o', c = 'blue')
plt.plot([0,55], [0,55], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Regression with MLP (Training Set)')
plt.ylim(0, 55)
plt.xlim(0, 55)
plt.axis(True)
plt.savefig("mlp-regr-regul-training.png")
plt.show()

print("Correlation Coefficient in training set: %.4f" % np.corrcoef(np.transpose(train_predictions), np.transpose(train_y))[0,1])